In [1]:
import numpy as np
np.set_printoptions(precision=4,suppress=True)
from scipy.linalg import block_diag


In [2]:
# Pauli matrices
I  = np.array([[ 1, 0],
               [ 0, 1]])
Sx = np.array([[ 0, 1],
               [ 1, 0]])
Sy = np.array([[ 0,-1j],
               [1j, 0]])
Sz = np.array([[ 1, 0],
               [ 0,-1]])

# Hadamard matrix
H = (1/np.sqrt(2))*np.array([[ 1, 1],
                             [ 1,-1]])

# Phase matrix
S = np.array([[ 1, 0],
              [ 0,1j]])

# single qubit basis states |0> and |1>
q0 = np.array([[1],
               [0]])
q1 = np.array([[0],
               [1]])

# Projection matrices |0><0| and |1><1|
P0  = np.dot(q0,q0.conj().T)
P1  = np.dot(q1,q1.conj().T)


# Rotation matrices as a function of theta, e.g. Rx(theta), etc.
Rx = lambda theta : np.array([[    np.cos(theta/2),-1j*np.sin(theta/2)],
                              [-1j*np.sin(theta/2),    np.cos(theta/2)]])
Ry = lambda theta : np.array([[    np.cos(theta/2),   -np.sin(theta/2)],
                              [    np.sin(theta/2),    np.cos(theta/2)]])
Rz = lambda theta : np.array([[np.exp(-1j*theta/2),                0.0],
                              [                0.0, np.exp(1j*theta/2)]])



In [3]:
# CNOTij, where i is control qubit and j is target qubit
CNOT10 = np.kron(P0,I) + np.kron(P1,Sx) # control -> q1, target -> q0
CNOT01 = np.kron(I,P0) + np.kron(Sx,P1) # control -> q0, target -> q1

SWAP   = block_diag(1,Sx,1)

In [4]:
# See DOI: 10.1103/PhysRevX.6.031007
# Here, we use parameters given for H2 at R=0.25A
g0 = +2.1868
g1 = +0.5449
g2 = -1.2870
g3 = +0.6719
g4 = +0.0798
g5 = +0.0798

nuclear_repulsion = 0.7055696146*3

In [5]:
Hmol = (g0 * np.kron( I, I) + # g0 * I
        g1 * np.kron( I,Sz) + # g1 * Z0
        g2 * np.kron(Sz, I) + # g2 * Z1
        g3 * np.kron(Sz,Sz) + # g3 * Z0Z1
        g4 * np.kron(Sy,Sy) + # g4 * Y0Y1
        g5 * np.kron(Sx,Sx))  # g5 * X0X1

In [6]:
print(Hmol)

[[ 2.1166+0.j  0.    +0.j  0.    +0.j  0.    +0.j]
 [ 0.    +0.j -0.317 +0.j  0.1596+0.j  0.    +0.j]
 [ 0.    +0.j  0.1596+0.j  3.3468+0.j  0.    +0.j]
 [ 0.    +0.j  0.    +0.j  0.    +0.j  3.6008+0.j]]


In [40]:
electronic_energy = np.linalg.eigvalsh(Hmol)[0] # take the lowest value
print("Classical diagonalization: {:+2.8} Eh".format(electronic_energy + nuclear_repulsion))
print("Exact (from G16):          {:+2.8} Eh".format(-1.1457416808))

Classical diagonalization: +1.7927696 Eh
Exact (from G16):          -1.1457417 Eh


In [41]:
# initial basis, state is |00>
psi0 = np.zeros((4,1))
psi0[0] = 1
print(psi0)

[[1.]
 [0.]
 [0.]
 [0.]]


In [42]:
# put in |01> state with Sx operator on q0
psi0 = np.dot(np.kron(I,Sx),psi0)
print(psi0)

[[0.]
 [1.]
 [0.]
 [0.]]


In [43]:
def expected(theta,ansatz,Hmol,psi0):
    circuit = ansatz(theta[0])
    psi = np.dot(circuit,psi0)
    return np.real(np.dot(psi.conj().T,np.dot(Hmol,psi)))[0,0]

In [44]:
theta


0.0

In [45]:
from scipy.linalg import expm
from scipy.optimize import minimize

# our UCC ansatz, not yet represented in terms of quantum gates
ansatz = lambda theta: expm(-1j*np.array([theta])*np.kron(Sy,Sx))

# initial guess for theta
theta  = [0.0]
result = minimize(expected,theta,args=(ansatz,Hmol,psi0))
theta  = result.x[0]
val    = result.fun

print("Lazy VQE: ")
print("  [+] theta:  {:+2.8} deg".format(theta))
print("  [+] energy: {:+2.8} Eh".format(val + nuclear_repulsion))

Lazy VQE: 
  [+] theta:  -0.043452772 deg
  [+] energy: +1.7927696 Eh


In [48]:
ansatz = lambda theta: (np.dot(np.dot(np.kron(-Ry(np.pi/2),Rx(np.pi/2)),
                        np.dot(CNOT10, 
                        np.dot(np.kron(I,Rz(theta)),
                               CNOT10))),
                               np.kron(Ry(np.pi/2),-Rx(np.pi/2))))


In [49]:
theta  = [0.0]
result = minimize(expected,theta,args=(ansatz,Hmol,psi0))
theta  = result.x[0]
val    = result.fun

print("VQE: ")
print("  [+] theta:  {:+2.8} deg".format(theta))
print("  [+] energy: {:+2.8} Eh".format(val + nuclear_repulsion))

VQE: 
  [+] theta:  +3.0546895 deg
  [+] energy: +1.7927696 Eh


In [51]:
def projective_expected(theta,ansatz,psi0):
    # this will depend on the hard-coded Hamiltonian + coefficients
    circuit = ansatz(theta[0])
    psi = np.dot(circuit,psi0)
    
    # for 2 qubits, assume we can only take Pauli Sz measurements (Sz \otimes I)
    # we just apply the right unitary for the desired Pauli measurement
    measureZ = lambda U: np.dot(np.conj(U).T,np.dot(np.kron(Sz,I),U))
    
    energy = 0.0
    
    # although the paper indexes the Hamiltonian left-to-right (0-to-1) 
    # qubit-1 is always the top qubit for us, so the tensor pdt changes
    # e.g. compare with the "exact Hamiltonian" we explicitly diagonalized
    
    # <I1 I0>
    energy += g0 # it is a constant

    # <I1 Sz0>
    U = SWAP
    energy += g1*np.dot(psi.conj().T,np.dot(measureZ(U),psi))

    # <Sz1 I0>
    U = np.kron(I,I)
    energy += g2*np.dot(psi.conj().T,np.dot(measureZ(U),psi))

    # <Sz1 Sz0>
    U = CNOT01
    energy += g3*np.dot(psi.conj().T,np.dot(measureZ(U),psi))

    # <Sx1 Sx0>
    U = np.dot(CNOT01,np.kron(H,H))
    energy += g4*np.dot(psi.conj().T,np.dot(measureZ(U),psi))

    # <Sy1 Sy0>
    U = np.dot(CNOT01,np.kron(np.dot(H,S.conj().T),np.dot(H,S.conj().T)))
    energy += g5*np.dot(psi.conj().T,np.dot(measureZ(U),psi))

    return np.real(energy)[0,0]

In [52]:
theta  = [0.0]
result = minimize(projective_expected,theta,args=(ansatz,psi0))
theta  = result.x[0]
val    = result.fun

print("VQE: ")
print("  [+] theta:  {:+2.8} deg".format(theta))
print("  [+] energy: {:+2.8} Eh".format(val + nuclear_repulsion))

VQE: 
  [+] theta:  +3.0546895 deg
  [+] energy: +1.7927696 Eh


In [80]:
# This function prints more details
def projective_expected_detail(theta,ansatz,psi0):
    
    print('Current VQE Iteration')
    # this will depend on the hard-coded Hamiltonian + coefficients
    circuit = ansatz(theta[0])
    psi = np.dot(circuit,psi0)
    
    # for 2 qubits, assume we can only take Pauli Sz measurements (Sz \otimes I)
    # we just apply the right unitary for the desired Pauli measurement
    measureZ = lambda U: np.dot(np.conj(U).T,np.dot(np.kron(Sz,I),U))
    
    energy = 0.0
    
    # although the paper indexes the Hamiltonian left-to-right (0-to-1) 
    # qubit-1 is always the top qubit for us, so the tensor pdt changes
    # e.g. compare with the "exact Hamiltonian" we explicitly diagonalized
    
    # <I1 I0>
    energy += g0 # it is a constant

    # <I1 Sz0>
    U = SWAP
    Eg1 = g1*np.dot(psi.conj().T,np.dot(measureZ(U),psi))
    energy += Eg1
    print('g1 =', Eg1)

    # <Sz1 I0>
    U = np.kron(I,I)
    Eg2 = g2*np.dot(psi.conj().T,np.dot(measureZ(U),psi))
    energy += Eg2
    print('g2 =', Eg2)
    
    # <Sz1 Sz0>
    U = CNOT01
    Eg3 = g3*np.dot(psi.conj().T,np.dot(measureZ(U),psi))
    energy += Eg3
    print('g3 =', Eg3)
    
    # <Sx1 Sx0>
    U = np.dot(CNOT01,np.kron(H,H))
    Eg4 = g4*np.dot(psi.conj().T,np.dot(measureZ(U),psi))
    energy += Eg4
    print('g4 =', Eg4)
    
    # <Sy1 Sy0>
    U = np.dot(CNOT01,np.kron(np.dot(H,S.conj().T),np.dot(H,S.conj().T)))
    Eg5 = g5*np.dot(psi.conj().T,np.dot(measureZ(U),psi))
    energy += Eg5
    print('g5 =', Eg5)
    print("\n")

    return np.real(energy)[0,0]

In [81]:
theta  = [0.0]

result = minimize(projective_expected_detail,theta,args=(ansatz,psi0))
theta  = result.x[0]
val    = result.fun


print("VQE: ")
print("  [+] theta:  {:+2.8} deg".format(theta))
print("  [+] energy: {:+2.8} Eh".format(val + nuclear_repulsion))

Current VQE Iteration
g1 = [[0.5449+0.j]]
g2 = [[1.287-0.j]]
g3 = [[-0.6719+0.j]]
g4 = [[-0.+0.j]]
g5 = [[-0.-0.j]]


Current VQE Iteration
g1 = [[0.5449+0.j]]
g2 = [[1.287-0.j]]
g3 = [[-0.6719+0.j]]
g4 = [[-0.+0.j]]
g5 = [[-0.-0.j]]


Current VQE Iteration
g1 = [[0.5449+0.j]]
g2 = [[1.287-0.j]]
g3 = [[-0.6719+0.j]]
g4 = [[-0.+0.j]]
g5 = [[-0.-0.j]]


Current VQE Iteration
g1 = [[0.538+0.j]]
g2 = [[1.2706-0.j]]
g3 = [[-0.6719+0.j]]
g4 = [[-0.0127+0.j]]
g5 = [[-0.0127+0.j]]


Current VQE Iteration
g1 = [[0.538+0.j]]
g2 = [[1.2706-0.j]]
g3 = [[-0.6719+0.j]]
g4 = [[-0.0127+0.j]]
g5 = [[-0.0127+0.j]]


Current VQE Iteration
g1 = [[0.538+0.j]]
g2 = [[1.2706-0.j]]
g3 = [[-0.6719+0.j]]
g4 = [[-0.0127+0.j]]
g5 = [[-0.0127+0.j]]


Current VQE Iteration
g1 = [[0.3804+0.j]]
g2 = [[0.8985-0.j]]
g3 = [[-0.6719+0.j]]
g4 = [[-0.0571+0.j]]
g5 = [[-0.0571+0.j]]


Current VQE Iteration
g1 = [[0.3804+0.j]]
g2 = [[0.8985-0.j]]
g3 = [[-0.6719+0.j]]
g4 = [[-0.0571+0.j]]
g5 = [[-0.0571+0.j]]


Current VQE It